# 01 ws

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
(X_train, _), (X_test, _) = fashion_mnist.load_data()

print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train[0][:, 7:20]

### Visualize

In [ ]:
img = X_train[1]
plt.imshow(img, cmap=plt.cm.gray_r) # 

plt.show()

In [ ]:
def visual_multi(i):

    nplots = 16
    fig = plt.figure(figsize=(12, 3)) #
    for j in range(nplots):

        plt.subplot(2, 8, j+1)
        plt.imshow(X_train[i+j], cmap=plt.cm.gray_r)

        plt.xticks([]); plt.yticks([])

    plt.show()

visual_multi(0)

In [ ]:
# X_train = X_train.astype('float32') / 255.0
# X_test = X_test.astype('float32') / 255.0

X_train = X_train / 255.0          #  ONCE
X_test = X_test / 255.0

In [ ]:
np.min(X_train), np.max(X_train)

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Reshape #
from tensorflow.keras import Model, Sequential

encoding_dim = 36 

class Autoencoder(Model):
    def __init__(self, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoding_dim = encoding_dim   
        self.encoder = Sequential([
#             InputLayer(input_shape=(28,28)),
            Flatten(),
            Dense(encoding_dim, activation='relu'),
        ])
        self.decoder = Sequential([
            Dense(784, activation='sigmoid'),
            Reshape((28, 28))
        ])
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(encoding_dim)

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(X_train, X_train, epochs=20,
                batch_size=256,
                validation_data=(X_test, X_test))

In [ ]:
autoencoder.encoder.summary()

In [ ]:
autoencoder.decoder.summary()

In [ ]:
encoded_imgs = autoencoder.encoder(X_test).numpy() 
print(encoded_imgs.shape)  

In [ ]:
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()  
decoded_imgs.shape

In [ ]:
plt.imshow(encoded_imgs[0].reshape(6,6), cmap=plt.cm.gray_r)

In [ ]:
plt.imshow(decoded_imgs[0], cmap=plt.cm.gray_r)

In [ ]:
n = 8
plt.figure(figsize=(14-2, 5.8+2.7)) 
for i in range(n):
    ax = plt.subplot(3, n, i + 1)
    plt.title("original")
    plt.imshow(X_test[i], cmap=plt.cm.gray_r)
    plt.xticks([]); plt.yticks([])

    bx = plt.subplot(3, n, n+i+1)
    plt.title("Encoded")
    plt.imshow(encoded_imgs[i].reshape(6,6), cmap=plt.cm.gray_r)

    plt.xticks([]); plt.yticks([])

    cx = plt.subplot(3, n, 2*n + i+ 1)
    plt.title("Reconstructed")
    plt.imshow(decoded_imgs[i], cmap=plt.cm.gray_r)
    plt.xticks([]); plt.yticks([])

plt.show()

# ws02 Denoise

In [ ]:
(X_train, _), (X_test, _) = fashion_mnist.load_data()

print(X_train.shape)

X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = np.expand_dims(X_train, -1) # 
X_test = np.expand_dims(X_test, -1)

print(X_train.shape)

In [ ]:
noise_factor = 0.2
X_train_noisy = X_train + noise_factor * np.random.normal(loc=0.0, 
                                                          scale=1.0, size=X_train.shape) 
X_test_noisy = X_test + noise_factor * np.random.normal(loc=0.0, 
                                                        scale=1.0, size=X_test.shape) 

X_train_noisy = np.clip(X_train_noisy, 0., 1.)
X_test_noisy = np.clip(X_test_noisy, 0., 1.)

X_train.shape

In [ ]:
img = X_train_noisy[1]
plt.imshow(img, cmap=plt.cm.gray_r) 
plt.show()

### Model

In [ ]:

from tensorflow.keras.layers import InputLayer,Conv2D, Conv2DTranspose 

class Denoise(Model):
    def __init__(self):
        super(Denoise, self).__init__()
        self.encoder = Sequential([

            Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
            Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

        self.decoder = Sequential([
            Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
            Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
            Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Denoise()

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder.fit(X_train_noisy, X_train, epochs=10,
                batch_size=128,
                validation_data=(X_test, X_test) )

In [ ]:
autoencoder.encoder.summary()

In [ ]:
autoencoder.decoder.summary()

In [ ]:
encoded_imgs = autoencoder.encoder(X_test_noisy).numpy() 
print(encoded_imgs.shape)

decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()
decoded_imgs.shape

### Visual

In [ ]:
n = 8
plt.figure(figsize=(12, 4))
for i in range(n):

    # display original + noise
    ax = plt.subplot(2, n, i+1)
    plt.title("Original+noise")
    plt.imshow(np.squeeze(X_test_noisy[i]), cmap=plt.cm.gray_r)
    plt.imshow(X_test_noisy[i], cmap=plt.cm.gray_r)
    plt.xticks([]); plt.yticks([])

    # display reconstruction
    bx = plt.subplot(2, n, i + n + 1)
    plt.title("Reconstructed")
    plt.imshow(np.squeeze(decoded_imgs[i]), cmap=plt.cm.gray_r)
    plt.xticks([]); plt.yticks([])

plt.show()